In [1]:
import math
import numpy
# load and prepare the dataset
dataset = numpy.loadtxt('data/data learning/learning.txt', delimiter=' ', dtype=int)
X = dataset[:, :-4]
Y = dataset[:,-4:]

numpy.argmax(Y, axis = 1)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)

In [2]:
# Sample Multilayer Perceptron Neural Network in Keras
from keras.models import Sequential
from keras.layers import Dense
# 1. define the network
model = Sequential()
model.add(Dense(550, input_dim=768, activation='sigmoid'))
model.add(Dense(400, input_dim=550, activation='sigmoid'))
model.add(Dense(150, input_dim=400, activation='sigmoid'))
model.add(Dense(4, input_dim=150, activation='softmax'))
# 2. compile the network
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# 3. fit the network
history = model.fit(X, Y, epochs=1400, batch_size=50, verbose=0)
# 4. evaluate the network
loss, accuracy = model.evaluate(X, Y)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
# 5. make predictions
probabilities = model.predict(X)
# print(probabilities)
predictions = numpy.argmax(probabilities, axis=1)
print(predictions)
accuracy = numpy.mean(predictions == numpy.argmax(Y, axis=1))
print("Prediction Accuracy: %.2f%%" % (accuracy*100))

Using TensorFlow backend.


128/128 [==============================] - 0s 244us/step

Loss: 0.12, Accuracy: 98.44%
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
Prediction Accuracy: 98.44%


In [3]:
    # serialize model to JSON
    model_json = model.to_json()
    with open("hasil learning/model " + str(accuracy * 100) + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("hasil learning/model " + str(accuracy * 100) + ".h5")
    print("Saved model to disk")

Saved model to disk


# 1. define the network
model = Sequential()
model.add(Dense(500, input_dim=768, activation='sigmoid'))
model.add(Dense(60, input_dim=500, activation='sigmoid'))
model.add(Dense(4, input_dim=250, activation='softmax'))
# 2. compile the network
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# 3. fit the network
history = model.fit(X, Y, epochs=500, batch_size=50, verbose=0)
# 4. evaluate the network
loss, accuracy = model.evaluate(X, Y)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
# 5. make predictions
probabilities = model.predict(X)
# print(probabilities)
predictions = numpy.argmax(probabilities, axis=1)
print(predictions)
accuracy = numpy.mean(predictions == numpy.argmax(Y, axis=1))
print("Prediction Accuracy: %.2f%%" % (accuracy*100))

if (accuracy*100) > 96:
    # serialize model to JSON
    model_json = model.to_json()
    with open("hasil learning/model " + str(accuracy * 100) + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("hasil learning/model " + str(accuracy * 100) + ".h5")
    print("Saved model to disk")

In [25]:
# load model
from keras.models import model_from_json

model_json = open('model.json', 'r')
loaded = model_from_json(model_json.read())
model_json.close()
loaded.load_weights('model.h5')
loaded.compile(loss='mse', optimizer='sgd', metrics=['accuracy'])
score = loaded.predict(numpy.array([X[5]]))
print(numpy.round(score[0,0]))

2.0
